In [1]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.jmt.*
import java.nio.file.Paths
import java.net.URI
GlobalConstants.setVerbose(VerboseLevel.STD);

In [2]:
// Block 1: Create network and nodes
val model = Network("M/G/1")
val source = Source(model, "Source")
val queue = Queue(model, "Queue", SchedStrategy.FCFS)
val sink = Sink(model, "Sink")

In [3]:
// Block 2: Create job classes
val jobclass1 = OpenClass(model, "Class1")
val jobclass2 = OpenClass(model, "Class2")

In [4]:
// Set arrival processes
source.setArrival(jobclass1, Exp(0.5))  // Arrival rate = 0.5
source.setArrival(jobclass2, Exp(0.5))  // Arrival rate = 0.5

In [5]:
import jline.examples.java.GettingStarted
// Set service processes
// Class1: Erlang distribution with mean=1, SCV=1/3
queue.setService(jobclass1, Erlang.fitMeanAndSCV(1.0, 1.0/3.0))

// Class2: Trace-driven service using APH fitting
val fileURI = jline.examples.java.GettingStarted::class.java.getResource("/example_trace.txt").toURI()
val fileName = Paths.get(fileURI).toString()
queue.setService(jobclass2, Replayer(fileName).fitAPH())

In [6]:
// Block 3: Define topology using routing matrix
val P = model.initRoutingMatrix()
P.set(jobclass1, Network.serialRouting(source, queue, sink))
P.set(jobclass2, Network.serialRouting(source, queue, sink))
model.link(P)

In [7]:
// Block 4: Solve using JMT solver and print results
val avgTable = JMT(model, "seed", 23000).avgTable
avgTable.print()

JMT Model: /tmp/workspace/jsim/17907634671192175435/jmodel.jsim
JMT [method: default, lang: java, env: 17.0.15] completed in 0.763275s.
Station  JobClass  QLen        Util        RespT       ResidT      ArvR        Tput        
-------------------------------------------------------------------------------------------
Source   Class1    0           0           0           0           0           0.50715     
Source   Class2    0           0           0           0           0           0.49142     
Queue    Class1    0.83239     0.47636     1.75038     1.75038     0.50715     0.50714     
Queue    Class2    0.40274     0.05027     0.83093     0.83093     0.49142     0.49138     
-------------------------------------------------------------------------------------------
